## Scrapping images of cetaceans from Wikipedia page

In [1]:
import pandas as pd

In [29]:
path_1 = "data/HappyWhales/location_cetacea_full_info.csv"
path_2 = "data/HappyWhales/obis_seamap_custom_part1_points_csv.csv"

# columns_to_keep_df1 = COLUMNS_TO_KEEP_1
# columns_to_keep_df2 = COLUMNS_TO_KEEP_2 

MAPPING = {
            'scientific': 'scientific_name',
            'common': 'common_name',
            "sp_code" : 'species_name',
            'tsn': 'itis_tsn',
            'count': 'individual_count',
            'obs_date': 'event_date',
            'oid' : 'organism_id'
            }

column_mapping = MAPPING

In [ ]:
df_location_1 = pd.read_csv(path_1, encoding="utf-8")
#df_location_1['organism_id'] = df_location_1['organism_id'].str.extract(r'(\d+)$')
dataset_id = df_location_1["dataset_id"].unique()

df_location_2 = pd.read_csv(path_2, low_memory=False)
#df_location_2 = df_location_2[~df_location_2["dataset_id"].isin(dataset_id)][columns_to_keep_df2]

df_location_2 = df_location_2.rename(columns=column_mapping)

df_location = pd.concat([df_location_1, df_location_2], ignore_index=True)

species_list = df_location['species_name'].dropna().unique()
species_list

array(['Humpback Whale', 'Northern bottlenose whale', 'Franciscana',
       'Gray Whale', "Arnoux's Beaked Whale", 'Killer Whale',
       "Dall's Porpoise", 'Fin Whale', 'Harbor Porpoise', 'Minke Whale',
       'Atlantic White-sided Dolphin', 'Blue Whale',
       "Baird's Beaked Whale", 'Bottlenose Dolphin', 'Dwarf sperm whale',
       "Bryde's whale", 'Atlantic Spotted Dolphin', 'False Killer Whale',
       'Long-finned Pilot Whale', 'North Atlantic Right Whale',
       'Hourglass Dolphin', "Commerson's Dolphin",
       "Cuvier's beaked whale", "Haviside's Dolphin",
       'Indian humpback dolphin', "Omurai's Whale", 'Common minke whale',
       'Beluga', 'Bowhead', "Cuvier's Beaked Whale",
       'Northern Right Whale Dolphin', 'Melon-headed Whale',
       "Hector's Dolphin", 'Dusky Dolphin', 'Black Dolphin',
       "Layard's beaked whale", 'Clymene Dolphin',
       "Blainville's Beaked Whale", "Burmeister's porpoise",
       "Gervais' beaked whale", 'Amazon River Dolphin',
       'I

In [2]:
IDs_and_names = pd.read_csv("images_and_sounds/aphiaIDs_and_names.csv").set_index("AphiaID")
IDs_and_names

,Description,Common name
AphiaID,,
105740,Alopias,Thresher shark
105835,Alopias superciliosus,Bigeye thresher shark
105836,Alopias vulpinus,common thresher shark
231390,Arctocephalus,NaN
231435,Arctocephalus australis,South American fur seal
...,...,...
255003,Zalophus californianus,California sea lion
255004,Zalophus japonicus,Japanese sea lion
255005,Zalophus wollebaeki,Galápagos sea lion


In [33]:
import requests, os
from tqdm import tqdm

# Set the folder name where images will be stored
my_folder = 'wiki_images'
os.makedirs(my_folder, exist_ok=True)

# Base URL for Wikipedia API
query = 'http://en.wikipedia.org/w/api.php?action=query&prop=pageimages&format=json&piprop=original&titles='

# Wikipedia article to download the main image from
my_list = species_list#IDs_and_names["Common name"].dropna().tolist()

# Define User-Agent header to respect Wikipedia's user-client policy :,)
HEADERS = {
    "User-Agent": "WhereWereWhales: cetaceans images from Wikipedia"
}


def get_image_url(partial_url):
    """ Gets the main image URL from a Wikipedia article """
    try:
        api_res = requests.get(query + partial_url, headers=HEADERS).json()
        first_part = api_res['query']['pages']
        for key, value in first_part.items():
            if 'original' in value:
                return value['original']['source']
    except Exception as exc:
        print(exc)
        print("Partial URL: " + partial_url)
    return None


def download_image(the_url, the_page):
    res = requests.get(the_url, headers=HEADERS) 
    res.raise_for_status()
    
    if res.status_code != 200:
        print(f"Error scraping {the_page}: {res.status_code}")

    # Get the correct file extension
    file_ext = '.' + the_url.split('.')[-1].lower()
    image_file = os.path.join(my_folder, os.path.basename(the_page + file_ext))

    # Save the image
    with open(image_file, 'wb') as f:
        for chunk in res.iter_content(100000):
            f.write(chunk)

    #print(f"Downloaded: {image_file}")

# Download images from Wikipedia articles
for the_page in tqdm(my_list, desc="Scraping Wikipedia images", unit="pages"):
    the_url = get_image_url(the_page)
    if the_url:
        #print(f"Downloading an image of a magnificent {the_page}...")
        download_image(the_url, the_page)
    else:
        print(f"No image found for {the_page} :(")

print("All done!")


Scraping Wikipedia images:   2%|▏         | 1/46 [00:28<21:07, 28.17s/pages]

No image found for Humpback Whale :(


Scraping Wikipedia images:   7%|▋         | 3/46 [00:29<04:53,  6.81s/pages]

No image found for Franciscana :(


Scraping Wikipedia images:   9%|▊         | 4/46 [00:30<02:59,  4.28s/pages]

...

No image found for Amazon River Dolphin :(


Scraping Wikipedia images:  98%|█████████▊| 45/46 [00:54<00:01,  1.43s/pages]

No image found for North Pacific Right Whale :(


Scraping Wikipedia images: 100%|██████████| 46/46 [00:55<00:00,  1.21s/pages]

All done!
